<a href="https://colab.research.google.com/github/ajmainn/cvpr_face_detection/blob/main/FaceDetection_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
import glob as gb
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
data_path = '/content/drive/MyDrive/Projects/Dataset'

In [12]:
IMG_SIZE = 256
BATCH_SIZE = 8

In [13]:
categories = []
class_count = []
no_of_samples = 0

for f in os.listdir(data_path):
  files = gb.glob(str(data_path + '//' + f + '/*'))
  categories.append(f)
  class_count = len(files)
  no_of_samples += len(files)

In [ ]:
print("Subfolder names (Class names):", categories)
print("Number of samples in each class:", class_count)
print("Total number of samples:", no_of_samples)

Subfolder names (Class names): ['ajmain', 'Afif', 'Avizit', 'Arnob', 'Abir', 'Alvi', 'Apurba', 'ASHHAB', 'Ashiq', 'Arshad', 'Hasnatur', 'Khushbu', 'Maishara', 'Jahin', 'Farhan', 'Hasib', 'Dristi', 'GM ABUBAKAR SIDDIQ', 'Ishmam', 'Kowshik', 'Rafsan', 'MRIDUL', 'Pranto', 'Prachurjo', 'Rassel', 'Ramisa', 'Nishat', 'Nipa Rahman', 'Mehrab', 'MAYSHA', 'Rizon', 'Saif', 'Sanim', 'Sayem', 'Rokon', 'Sadi', 'Rono', 'Shaila', 'Sakib', 'Ridoy', 'ShuvoSaha', 'Shatabdi', 'Sudipta', 'Sumaiya', 'Sintheia', 'Shaivik', 'shohidul', 'Siam', 'SHUVO', 'Shakibul', 'Tahsin', 'Talha', 'Taj', 'Zarin', 'Tahsin Kabir', 'SUN']
Number of samples in each class: 5
Total number of samples: 434


In [14]:
train_gen = ImageDataGenerator(
  rotation_range = 20,
  width_shift_range = 0.2,
  height_shift_range = 0.2,
  horizontal_flip = True,
  zoom_range = 0.1,
  preprocessing_function = tf.keras.applications.mobilenet_v2.preprocess_input,
  validation_split = 0.2
)

In [15]:
train_batch = train_gen.flow_from_directory(
  directory = data_path,
  target_size = (IMG_SIZE, IMG_SIZE),
  batch_size = BATCH_SIZE,
  class_mode = 'sparse',
  subset = 'training'
)
valid_batch = train_gen.flow_from_directory(
  directory = data_path,
  target_size = (IMG_SIZE, IMG_SIZE),
  batch_size = BATCH_SIZE,
  class_mode = 'sparse',
  subset = 'validation'
)

Found 326 images belonging to 55 classes.
Found 66 images belonging to 55 classes.


In [16]:
img_shape = (IMG_SIZE, IMG_SIZE) + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, include_top = False)
base_model.summary()

<ipython-input-16-5b371d99d745>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, include_top = False)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "mobilenetv2_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 128, 128, 32)   │            864 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 128, 128, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 128, 128, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 128, 128, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 128, 128, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 128, 128, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 128, 128, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 128, 128, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 128, 128, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 128, 128, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 128, 128, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 129, 129, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 64, 64, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise_BN 

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 2,223,872 (8.48 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [17]:
from re import X
inputs = tf.keras.Input(shape = (IMG_SIZE, IMG_SIZE, 3))
X = base_model(inputs, training = False)
X = tf.keras.layers.GlobalAveragePooling2D()(X)
X = tf.keras.layers.Dense(2048, activation = 'relu')(X)
X = tf.keras.layers.Dropout(0.4)(X)
X = tf.keras.layers.Dense(1024, activation = 'relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(512, activation = 'relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
outputs = tf.keras.layers.Dense(len(categories), activation = 'softmax')(X)

model = tf.keras.Model(inputs, outputs)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ mobilenetv2_1.00_224 (Functional)    │ (None, 8, 8, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 2048)                │       2,623,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1024)                │       2,098,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         524,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 56)                  │          28,728 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 7,533,176 (28.74 MB)

 Trainable params: 7,499,064 (28.61 MB)

 Non-trainable params: 34,112 (133.25 KB)

In [18]:
model.compile(
    loss = tf.keras.losses.sparse_categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
    metrics = ['accuracy']
)

In [19]:
early_stopping = EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3)


In [20]:
h = model.fit(
    train_batch,
    epochs = 2,
    validation_data = valid_batch,
    callbacks = [early_stopping, checkpoint, reduce_lr]
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.0222 - loss: 4.3389
Epoch 1: val_accuracy improved from -inf to 0.03030, saving model to best_model.keras
41/41 ━━━━━━━━━━━━━━━━━━━━ 428s 9s/step - accuracy: 0.0220 - loss: 4.3354 - val_accuracy: 0.0303 - val_loss: 8.6691 - learning_rate: 0.0010
Epoch 2/2
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step - accuracy: 0.0299 - loss: 3.9271
Epoch 2: val_accuracy did not improve from 0.03030
41/41 ━━━━━━━━━━━━━━━━━━━━ 64s 591ms/step - accuracy: 0.0296 - loss: 3.9269 - val_accuracy: 0.0303 - val_loss: 8.9273 - learning_rate: 0.0010


In [21]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

from google.colab.output import eval_js
from IPython.display import display, Javascript
from base64 import b64decode
import cv2
import numpy as np
from PIL import Image
from io import BytesIO

import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array
from google.colab.patches import cv2_imshow
from keras.models import load_model
from PIL import Image

In [ ]:
def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      document.body.appendChild(div);
      div.appendChild(capture);

      const video = document.createElement('video');
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      video.srcObject = stream;
      await video.play();

      // Resize video output
      video.width = 320;
      video.height = 240;
      div.appendChild(video);

      // Wait for the user to click the capture button
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      stream.getTracks().forEach(t => t.stop());
      div.remove();

      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

# Capture photo and display
image_path = take_photo()

# Load the captured image
img = Image.open(image_path)
img_array = np.array(img)

# Now you can use the captured image for face detection and classification
cv2_imshow(img_array)


In [ ]:
model = load_model('best_model.keras')

# Load the label names (replace with your actual class names/subfolder names)
categories = ['ajmain', 'Afif', 'Avizit', 'Arnob', 'Abir', 'Alvi', 'Apurba', 'ASHHAB', 'Ashiq', 'Arshad', 'Hasnatur', 'Khushbu', 'Maishara', 'Jahin', 'Farhan', 'Hasib', 'Dristi', 'GM ABUBAKAR SIDDIQ', 'Ishmam', 'Kowshik', 'Rafsan', 'MRIDUL', 'Pranto', 'Prachurjo', 'Rassel', 'Ramisa', 'Nishat', 'Nipa Rahman', 'Mehrab', 'MAYSHA', 'Rizon', 'Saif', 'Sanim', 'Sayem', 'Rokon', 'Sadi', 'Rono', 'Shaila', 'Sakib', 'Ridoy', 'ShuvoSaha', 'Shatabdi', 'Sudipta', 'Sumaiya', 'Sintheia', 'Shaivik', 'shohidul', 'Siam', 'SHUVO', 'Shakibul', 'Tahsin', 'Talha', 'Taj', 'Zarin', 'Tahsin Kabir', 'SUN']

# Load OpenCV's Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Preprocess the image for model prediction
def preprocess_image(image, target_size):
    image = cv2.resize(image, target_size)  # Resize to target size
    image = img_to_array(image)  # Convert to array
    image = np.expand_dims(image, axis=0)  # Add batch dimension
    return image

# Function to detect faces and classify them
def detect_and_classify(image_array):
    # Convert the image to grayscale for face detection
    gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Loop through detected faces and classify each one
    for (x, y, w, h) in faces:
        # Extract the region of interest (the face) from the image
        face = image_array[y:y+h, x:x+w]

        # Preprocess the face for the model
        preprocessed_face = preprocess_image(face, target_size=(IMG_SIZE, IMG_SIZE))

        # Predict the class label using the pre-trained model
        prediction = model.predict(preprocessed_face)
        label_index = np.argmax(prediction, axis=1)[0]
        label = categories[label_index]

        # Draw a rectangle around the face and label it
        cv2.rectangle(image_array, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(image_array, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    return image_array

# Detect and classify faces in the captured image
output_image = detect_and_classify(img_array)

# Display the image with bounding boxes and labels
cv2_imshow(output_image)
